### 1. SparkSession
---
- 가장 먼저 해야할 것은 SparkContext라는 스파크 객체를 만들어야 한다.
- SparkContext를 만들기 위해서는 SparkSession이 있어야 한다.
- 그리고 json 형색으로 되어 있는 데이터를 읽어 데이터의 스키마를 출력시켜보자.

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
            .builder
            .appName('Python Spark SQL Basic Example')
            .config('spark.some.config.option', 'some-value')
            .getOrCreate())

# Create json file using spark
sc = spark.sparkContext

# Read json file
path = '/Users/yangws/Downloads/test.json'
testDF = spark.read.json(path)

# printSchema()로 json파일의 스키마 형태를 볼 수 있음
testDF.printSchema()

22/01/24 13:31:58 WARN Utils: Your hostname, yangwsui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.8 instead (on interface en7)
22/01/24 13:31:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/24 13:31:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- name: string (nullable = true)
 |-- number: long (nullable = true)



### SQL로 DataFrame 데이터 출력하기
---
- 데이터베이스 기능 중 'View'라는 가상의 테이블로 데이터프레임을 만들 수 있다.
- 가상 테이블을 만들기 위해 createOrReplaceTempView('name')을 수행하면 된다.

In [5]:
# 데이터프레임을 사용하는 임시 view 생성
testDF.createOrReplaceTempView('test')

# spark에서 제공하는 sql 메소드를 이용해 쿼리 보내기
# 쿼리문에서 test 테이블은 위에서 만든 view 테이블임
testNameDF = spark.sql("SELECT name FROM test")

# 만들어진 데이터프레임을 보기 위한 함수 show()
testNameDF.show()

+----+
|name|
+----+
|yang|
+----+



- View 테이블을 생성하는 메소드 중 Global Temporary View라는 것이 있다.
- 위에서 사용했던 일반적인 Temporary View 가상 테이블은 SparkSession이 종료되면 삭제된다.
- 하지만 모든 SparkSession들 간에 View 가상 테이블을 공유하게 만들고 싶다면 Global Temporary  View를 사용하면 된다.
- 한 가지 주의해야 할 점은 Global Temporary View 테이블을 만들고 SQL로 데이터를 추출할 때 테이블 이름 앞에 `global\_temp`라는 키워드를 붙여줘야 함을 잊지 말아야 한다.

In [3]:
# Load json file
path = '/Users/yangws/Downloads/test.json'
df = spark.read.json(path)

# Create Global Temporary View 
df.createGlobalTempView('test')

# 'global_temp' 키워드 꼭 붙이기
sqlDF = spark.sql('SELECT * FROM global_temp.test')
sqlDF.show()

+----+------+
|name|number|
+----+------+
|yang|   123|
+----+------+



- 부가적인 내용) json 형식의 데이터를 RDD로 만들어주고 데이터프레임으로 읽어오는 방식
---

In [6]:
# 데이터프레임은 RDD[String] 자료구조를 이용해 json 데이터셋을 데이터프레임으로 만들 수 있음
jsonStrings = ['{"name": "Yin", "address": {"city": "Columbus", "state": "Ohio"}}']

# json to RDD format
otherTestRDD = sc.parallelize(jsonStrings)

# Load json file
otherTest = spark.read.json(otherTestRDD)
otherTest.show()

+----------------+----+
|         address|name|
+----------------+----+
|{Columbus, Ohio}| Yin|
+----------------+----+



### 3. Spark DataFrame의 컬럼 접근
---
- Spark 내부에서 Dataset과 DataFrame의 차이는 Typed/Untyped 이다.
- Typed Dataset은 Dataset으로 받아오는 데이터의 형태를 미리 정의해놓은 것
- Untyped Dataset은 프로그램이 데이터의 형태를 추측해서 가져오는 것
- 이 두 개는 에러를 잡아내는 시간 측면에서도 차이점이 존재한다.

In [7]:
# Load json file
path = '/Users/yangws/Downloads/test.json'
df = spark.read.json(path)

# name 컬럼 SELECT
df.select('name').show()

+----+
|name|
+----+
|yang|
+----+



- 추가적으로 2개 이상의 컬럼을 추출하면서 기존의 컬럼에 연산을 더해 파생변수를 생성하여 추출하는 것도 가능하다.

In [10]:
# Load json file
path = '/Users/yangws/Downloads/test.json'
df = spark.read.json(path)

# name 컬럼 SELECT
df.select(df['name'], df['number'] + 100).show()

+----+--------------+
|name|(number + 100)|
+----+--------------+
|yang|           223|
+----+--------------+



- 다음은 특정 조건을 만족하는 데이터만 추출하는 filter()와 특정 컬럼으로 그룹핑해주는 groupBy() 메소드에 대한 예시 코드이다.

In [12]:
# Load json file
path = '/Users/yangws/Downloads/test.json'
df = spark.read.json(path)

# number가 20보다 큰 데이터만 추출
df.filter(df['number'] > 20).show()

# number 컬럼으로 그룹핑하고 데이터의 개수를 집계
df.groupBy('number').count().show()

+----+------+
|name|number|
+----+------+
|yang|   123|
+----+------+

+------+-----+
|number|count|
+------+-----+
|   123|    1|
+------+-----+



### DataFrame의 Schema를 프로그래밍스럽게 명시해보자
---
- 로드할 데이터(csv, txt, excel 등)의 여러가지 종류의 파일에는 스키마 즉, 컬럼명(헤더)가 기입되어 있지 않은 경우가 있다.
- 이럴 때는 프로그래머가 직접 명시해주어야 한다.
- 이러한 경우를 대비해 데이터파일 외부에 명시되어 있는 스키마를 가져와 데이터프레임의 헤더에 삽입해주는 코드를 만들어보자

In [17]:
from pyspark.sql.types import *

# Create SparkContext Object
sc = spark.sparkContext

# Load txt file
lines = sc.textFile('./Data/people.txt')
parts = lines.map(lambda x: x.split(','))

# Step 1 -> values 처리
# 각 라인을 tuple( , ) 형태로 convert
people = parts.map(lambda x: (x[0], x[1].strip())) # name에서 공백 strip

# Step 2 -> Schemas 처리
# 문자열로 인코딩된 스키마
schemaString = 'name age'

# schemaString 요소를 loop로 돌며 StructField로 만들기
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
# StructField 여러개가 있는 리스트를 StructType으로 만들기
schema = StructType(fields)

# Step 3 -> Value와 Schema를 활용해 DataFrame 생성
# 위에서 만든 Schema를 RDD의 Schema로 적용
schemaPeople = spark.createDataFrame(people, schema)

# View Table을 생성해 쿼리를 날려 데이터 출력하기
schemaPeople.createOrReplaceTempView('people')
results = spark.sql("SELECT * FROM people")
results.show()

+-------+---+
|   name|age|
+-------+---+
| Justin| 19|
|Michael| 29|
|   Andy| 30|
+-------+---+

